In [1]:
import syft as sy
import numpy as np
from syft.core.node.worker import Worker
from syft.core.node.new.user_code import SubmitUserCode
from syft.core.node.new.policy import SubmitUserPolicy

2023-03-10 19:44:19.635040: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.5/lib64:/usr/local/cuda-11.5/lib64:
2023-03-10 19:44:19.650175: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-10 19:44:20.092081: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.5/lib64:/usr/local/cuda-11.5/lib64:
2023-03-10 19:44:20.092235: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvrtc.so.11.1: cannot open shared 

In [2]:
worker = Worker()

> Starting Worker: Distracted Song - 0beb94bcd49143a9a12e6fea92a4b991 - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.task.oblv_service.OblvService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.DataSubjectMemberService'>]


In [3]:
client = sy.login(node=worker, email="info@openmined.org", password="changethis")

Logged into Distracted Song as <info@openmined.org>


In [4]:
import inspect
import sys
from IPython.core.magics.code import extract_symbols

def is_interpreter_jupyter() -> bool:
    return get_interpreter_module() == 'ipykernel.zmqshell'

def is_interpreter_colab() -> bool:
    return get_interpreter_module() == 'google.colab._shell'

def is_interpreter_standard() -> bool:
    return get_interpreter_module() == 'StandardInterpreter'

def get_interpreter_module() -> str:
    try:
        shell = get_ipython().__class__.__module__
        return shell
    except NameError:
        return "StandardInterpreter" # not sure    

def new_getfile(object):
    if not inspect.isclass(object):
        return inspect.getfile(object)
    
    # Lookup by parent module (as in current inspect)
    if hasattr(object, '__module__'):
        object_ = sys.modules.get(object.__module__)
        if hasattr(object_, '__file__'):
            return object_.__file__
    
    # If parent module is __main__, lookup by methods (NEW)
    for _, member in inspect.getmembers(object):
        if inspect.isfunction(member) and object.__qualname__ + '.' + member.__name__ == member.__qualname__:
            return inspect.getfile(member)
    else:
        raise TypeError('Source for {!r} not found'.format(object))

def get_code_from_class(policy):
    klasses = inspect.getmro(policy)[-2::-1]
    whole_str = ""
    for klass in klasses:
        if is_interpreter_jupyter():
            cell_code = "".join(inspect.linecache.getlines(new_getfile(klass)))
            class_code = extract_symbols(cell_code, klass.__name__)[0][0]
        else:
            class_code = inspect.getsource(klass)
        whole_str += class_code
    return whole_str

In [5]:
from syft.core.common.serde.serializable import serializable
from syft.core.node.new.user_code import ExactMatch, SingleExecutionExactOutput, OutputPolicy
from syft.core.node.new.action_object import ActionObject

class RepeatedCallPolicy():
    # from syft.core.node.new.action_object import ActionObject
    from typing import Dict
    from typing import Any
    
    def __init__(self, n_calls=1, downloadable_output_args=[]):
        self.n_calls = n_calls + 1
        self.downloadable_output_args = downloadable_output_args
        self.state = {"counts": 0}
        # super.__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)

    def __repr__(self) -> str:
        return f'RepeatedCallPolicy - {self.n_calls}/{self.downloadable_output_args}/{self.state}'

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, action_object) -> Dict[Any, Any]:
        results_dict = action_object.syft_action_data        
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                # results_dict[output_arg] = results_dict[output_arg].set_permission(READ, self.my_verify_key)
                results_dict[output_arg] = results_dict[output_arg]

            self.state["counts"] += 1
        else:
            return 
            
        return results_dict
    
    

In [6]:
policy = RepeatedCallPolicy(downloadable_output_args=['y'])
obj = ActionObject.from_obj({'y': [1,2,3]})
policy.apply_output(obj)

{'y': [1, 2, 3]}

In [7]:
import numpy as np
from syft.core.node.new.twin_object import to_action_object

x = np.array([1,2,3])
x_pointer = to_action_object(x)
client.api.services.action.save(x_pointer)

<class 'syft.core.node.new.response.SyftSuccess'>: <class 'syft.core.node.new.numpy.NumpyArrayObject'> saved

In [8]:
@sy.syft_function(
    input_policy=ExactMatch(x=x_pointer),
    # input_policy_init_args={'inputs':{'x':4}},
    # output_policy=SingleExecutionExactOutput,
    # output_policy_init_args={},
    output_policy=RepeatedCallPolicy,
    output_policy_init_args={"n_calls":1, 'downloadable_output_args':['y']},
    outputs=["y"]
)
def func(x):
    return {"y": x}


SubmitUserPolicy
('x',)


In [9]:
client.api.services.code.request_code_execution(func)

Noerr before code to usercode
['x']
code processing complete
Noerr before policy transformations
Noerr


```python
class Request:
  id: str = da386ab680f34d09b30d2bb5a5f6d279
  requesting_user_verify_key: str = 16ccb1e2bdabeae2e1705fbdfd3b8eac28827dd99535fe116e68b1b9fd808986
  approving_user_verify_key: str = None
  request_time: str = 2023-03-10 15:44:22
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 0beb94bcd49143a9a12e6fea92a4b991
  request_hash: str = "ab56f47a0a23e202c0f675d341ef37d88e4832217ac1ed88663c50ad4a028779"
  changes: str = [syft.core.node.new.request.UserCodeStatusChange]

```

In [10]:
client = sy.login(node=worker, email="info@openmined.org", password="changethis", cache=False)

Logged into Distracted Song as <info@openmined.org>


In [11]:
request = client.notifications[0].link
func = request.changes[0].link
result = func.unsafe_function(x=x_pointer)
# func.output_policy
final_result = request.accept_by_depositing_result(result)
final_result

<class 'syft.core.node.new.response.SyftSuccess'>: Request da386ab680f34d09b30d2bb5a5f6d279 changes applied

In [12]:
func.output_policy

```python
class UserPolicy:
  id: str = 8a5f8700ba6649eca581210bac6ff3ec
  user_verify_key: str = 16ccb1e2bdabeae2e1705fbdfd3b8eac28827dd99535fe116e68b1b9fd808986
  raw_code: str = "@serializable(recursive_serde=True)
class RepeatedCallPolicy():
    # from syft.core.node.new.action_object import ActionObject
    from typing import Dict
    from typing import Any
    
    def __init__(self, n_calls=1, downloadable_output_args=[]):
        self.n_calls = n_calls + 1
        self.downloadable_output_args = downloadable_output_args
        self.state = {"counts": 0}
        # super.__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)

    def __repr__(self) -> str:
        return f'RepeatedCallPolicy - {self.n_calls}/{self.downloadable_output_args}/{self.state}'

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, action_object) -> Dict[Any, Any]:
        results_dict = action_object.syft_action_data        
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                # results_dict[output_arg] = results_dict[output_arg].set_permission(READ, self.my_verify_key)
                results_dict[output_arg] = results_dict[output_arg]

            self.state["counts"] += 1
        else:
            return 
            
        return results_dict
"
  parsed_code: str = "@serializable(recursive_serde=True)
class RepeatedCallPolicy():
    # from syft.core.node.new.action_object import ActionObject
    from typing import Dict
    from typing import Any
    
    def __init__(self, n_calls=1, downloadable_output_args=[]):
        self.n_calls = n_calls + 1
        self.downloadable_output_args = downloadable_output_args
        self.state = {"counts": 0}
        # super.__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)

    def __repr__(self) -> str:
        return f'RepeatedCallPolicy - {self.n_calls}/{self.downloadable_output_args}/{self.state}'

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, action_object) -> Dict[Any, Any]:
        results_dict = action_object.syft_action_data        
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                # results_dict[output_arg] = results_dict[output_arg].set_permission(READ, self.my_verify_key)
                results_dict[output_arg] = results_dict[output_arg]

            self.state["counts"] += 1
        else:
            return 
            
        return results_dict
"
  signature: str = (downloadable_output_args)
  class_name: str = "RepeatedCallPolicy"
  unique_name: str = "user_func_RepeatedCallPolicy_16ccb1e2bdabeae2e1705fbdfd3b8eac28827dd99535fe116e68b1b9fd808986_c2b88b38b612f1aa9f2c57ab61c711a8332435b895ebddf906d2ba84c183a6b7"
  code_hash: str = "c2b88b38b612f1aa9f2c57ab61c711a8332435b895ebddf906d2ba84c183a6b7"
  status: str = UserPolicyStatus.SUBMITTED

```

In [13]:
client.api.services.code.func(x=x_pointer)

{'x': array([1, 2, 3])}


<class 'syft.core.node.new.response.SyftError'>: Failed to run. local variable 'kwargs' referenced before assignment

In [14]:
client.api.services.code.review(uid=func_id, approved=True)

NameError: name 'func_id' is not defined

In [ ]:
output_policy = client.api.services.policy.get_all()[0]


In [ ]:
client.api.services.code.func(x=x_pointer.id)

{'x': array([1, 2, 3])}


,key,type,id
0,y,"array([1, 2, 3])",ce5e0e35b4c549bbb33643f4bd14c919


In [ ]:
@sy.syft_function(
    input_policy=ExactMatch,
    input_policy_init_args={'inputs':{'x':4}},
    # output_policy=SingleExecutionExactOutput,
    # output_policy_init_args={},
    output_policy=output_policy.id,
    output_policy_init_args={"n_calls":1, 'downloadable_output_args':['y']},
    outputs=["y"]
)
def func_plus_one(x):
    return {"y": x + 1}


Output policy is
UserPolicy


In [ ]:
client.api.services.code.submit(func_plus_one)

id=<UID: f542fe3a20264fcdaecc8054cffc153b> user_verify_key=6c267347f99ca960dcbd0f5431e6ed0f7f4b7f8009723e19b54af2782f8bf9d3 raw_code='@serializable(recursive_serde=True)\nclass RepeatedCallPolicy():\n    from typing import Dict\n    from typing import Any\n    \n    def __init__(self, n_calls=1, downloadable_output_args=[]):\n        self.n_calls = n_calls + 1\n        self.downloadable_output_args = downloadable_output_args\n        self.state = {"counts": 0}\n        # super.__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)\n\n    def __repr__(self) -> str:\n        return f\'RepeatedCallPolicy - {self.n_calls}/{self.downloadable_output_args}/{self.state}\'\n\n    def public_state(self):\n        return self.state["counts"]\n        \n    def apply_output(self, results_dict, **kwargs) -> Dict[Any, Any]:\n        \n        if self.state["counts"] < self.n_calls:\n            for output_arg in self.downloadable_output_args:\n                # results_dict[out

<class 'syft.core.node.new.response.SyftSuccess'>: User Code Submitted

In [ ]:
client.api.services.policy.get_all()[0]

```python
class UserPolicy:
  id: str = f542fe3a20264fcdaecc8054cffc153b
  user_verify_key: str = 6c267347f99ca960dcbd0f5431e6ed0f7f4b7f8009723e19b54af2782f8bf9d3
  raw_code: str = "@serializable(recursive_serde=True)
class RepeatedCallPolicy():
    from typing import Dict
    from typing import Any
    
    def __init__(self, n_calls=1, downloadable_output_args=[]):
        self.n_calls = n_calls + 1
        self.downloadable_output_args = downloadable_output_args
        self.state = {"counts": 0}
        # super.__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)

    def __repr__(self) -> str:
        return f'RepeatedCallPolicy - {self.n_calls}/{self.downloadable_output_args}/{self.state}'

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, results_dict, **kwargs) -> Dict[Any, Any]:
        
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                # results_dict[output_arg] = results_dict[output_arg].set_permission(READ, self.my_verify_key)
                results_dict[output_arg] = results_dict[output_arg]

            self.state["counts"] += 1
        else:
            return 
            
        return results_dict
"
  parsed_code: str = "@serializable(recursive_serde=True)
class RepeatedCallPolicy():
    from typing import Dict
    from typing import Any
    
    def __init__(self, n_calls=1, downloadable_output_args=[]):
        self.n_calls = n_calls + 1
        self.downloadable_output_args = downloadable_output_args
        self.state = {"counts": 0}
        # super.__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)

    def __repr__(self) -> str:
        return f'RepeatedCallPolicy - {self.n_calls}/{self.downloadable_output_args}/{self.state}'

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, results_dict, **kwargs) -> Dict[Any, Any]:
        
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                # results_dict[output_arg] = results_dict[output_arg].set_permission(READ, self.my_verify_key)
                results_dict[output_arg] = results_dict[output_arg]

            self.state["counts"] += 1
        else:
            return 
            
        return results_dict
"
  signature: str = (downloadable_output_args)
  class_name: str = "RepeatedCallPolicy"
  unique_name: str = "user_func_RepeatedCallPolicy_6c267347f99ca960dcbd0f5431e6ed0f7f4b7f8009723e19b54af2782f8bf9d3_081a0eafec31fb60fb325a5e83ba0b8803fa2e03ad3f9649120a2e31686ac864"
  code_hash: str = "081a0eafec31fb60fb325a5e83ba0b8803fa2e03ad3f9649120a2e31686ac864"
  status: str = UserPolicyStatus.SUBMITTED

```

In [ ]:
func_plus_one

```python
class SubmitUserCode:
  id: str = None
  code: str = "@sy.syft_function(
    input_policy=ExactMatch,
    input_policy_init_args={'inputs':{'x':4}},
    # output_policy=SingleExecutionExactOutput,
    # output_policy_init_args={},
    output_policy=output_policy.id,
    output_policy_init_args={"n_calls":1, 'downloadable_output_args':['y']},
    outputs=["y"]
)
def func_plus_one(x):
    return {"y": x + 1}
"
  func_name: str = "func_plus_one"
  signature: str = (x)
  input_policy: str = id=<UID: b79395fd05cd4c68b1a6d8f3532647d2> inputs={} node_uid=None
  input_policy_init_args: str = {'inputs': {'x': 4}}
  output_policy: str = f542fe3a20264fcdaecc8054cffc153b
  output_policy_init_args: str = {'n_calls': 1, 'downloadable_output_args': ['y']}
  local_function: str = <function func_plus_one at 0x7f3225b0c700>
  input_kwargs: str = ['x']
  outputs: str = ['y']

```

In [ ]:
client.api.services.code.func

<function syft.core.node.new.api.generate_remote_function.<locals>.wrapper(*args, **kwargs)>

In [ ]:
client.

In [ ]:
func_id = client.api.services.code.get_all()[0]

ValidationError: 2 validation errors for UserCode
output_policy_state
  'utf-8' codec can't decode byte 0xca in position 100: invalid continuation byte (type=value_error.unicodedecode)
output_policy_state
  value is not a valid dict (type=type_error.dict)

In [ ]:
client.api.services.code.review(uid=func_id, approved=True)

: 

In [ ]:
client = sy.login(node=worker, email="info@openmined.org", password="changethis", cache=False)

Logged into Priceless Bengios as <info@openmined.org>


In [ ]:
client.api.services.code.func_plus_one(x=x_pointer.id)

: 

In [ ]:
client.api.services.code.get_all()

: 

In [ ]:
func_plus_one = client.api.services.code.get_all()[-1]

: 

In [ ]:
print(func_plus_one.output_policy.raw_code)

: 

In [ ]:
state = sy.deserialize(func_plus_one.output_policy_state, from_bytes=True, class_type=RepeatedCallPolicy)

: 

In [ ]:
state.public_state()

: 